# 全部打卡整合脚本


## 导入需要的Python库

In [1]:
# 必须导入，不导入没办法运行
import pandas as pd
import numpy as np
import os

## 载入所有名单(在此处修改项目数量以及名字)

In [2]:
# 可以修改，有多少项目就添加多少项目，项目名称需要与excel文件 除去后缀的 名称一致
# 项目名称要加英文单引号，不同项目之间要用逗号隔开
event_list = ['练字', '单词', '跑步', '光盘', '记账'] 

# excel文件的格式（默认为xlsx，如果是csv的话就改成.csv）
suffix = ".xlsx"

## 读取相应打卡数据

In [3]:
# 初始化一个字典，用于根据上面列表提供的项目读取对应的项目打卡分数数据
data_dict = {}
# 分别导入每个项目的excel数据

for item in event_list:
    item_name = item + suffix
    data_dict[item] = pd.read_excel(item_name)




## 创建整合全部打卡项目的名单的函数

In [4]:
def sum_name(dict):
    result_name = []
    num = 1
    for key in dict: # 遍历字典 data_dict 中的每个键
        events = dict[key] # 获取当前键对应的值（即数据帧），并将其赋值给 df
        if num == 1:
            for name in events["圈子昵称"]:
                result_name.append(name)
                
        else:
            for name in events["圈子昵称"]: # 遍历名单
                if name not in result_name:  # 判断：如果最终名单中没有这个人，就将这个人导入最终名单
                    result_name.append(name)
                
        num += 1
        
    return result_name

## 运行该函数，获得打卡名单总表

In [5]:
a = sum_name(data_dict) # 调用函数
len(a)

19

## 创建最终结果的DataFrame

In [6]:
final_df = pd.DataFrame(a) # 用最终名单列表的创建一个新的DataFrame
final_df

,0
0,1034111911兰斯
1,1024533380孙笑川
2,1044453937王八
3,1034343407王雷
4,1024191981张全蛋
5,1024102265陈楞宇
6,1034104114王晓为
7,1024100594彭泽军
8,1034103034梁左先
9,1044114514李田所


## 下面分别更改不同档次名单DataFrame的行index

In [8]:
data_dict_reindexed = {}

In [9]:
def reindex(dict):
    rein = "_reindexed"
    for key in dict:
        event_reindexed = key + rein
        tmp_df = dict[key]
        tmp_df = tmp_df.set_index(pd.Index(tmp_df["圈子昵称"]))
        data_dict_reindexed[event_reindexed] = tmp_df

In [10]:
reindex(data_dict)
data_dict_reindexed["跑步_reindexed"]

,圈子昵称,积分
圈子昵称,,
1024132849林林,1024132849林林,14
1044100120梁凌薇,1044100120梁凌薇,14
1034722522罗小黑,1034722522罗小黑,12
1044453125妃咲,1044453125妃咲,11
1054321380圣园未花,1054321380圣园未花,0


In [11]:
final_df_reindexed = final_df.set_index(pd.Index(final_df[0]))

final_df_reindexed

,0
0,
1034111911兰斯,1034111911兰斯
1024533380孙笑川,1024533380孙笑川
1044453937王八,1044453937王八
1034343407王雷,1034343407王雷
1024191981张全蛋,1024191981张全蛋
1024102265陈楞宇,1024102265陈楞宇
1034104114王晓为,1034104114王晓为
1024100594彭泽军,1024100594彭泽军
1034103034梁左先,1034103034梁左先


## 创建五个函数，分别计算每个人的打卡天数以及积分

### 给参与者统计每一个项目获得的分数的函数

In [12]:
def 单独项目赋分(项目, df):
    积分 = [] # 创建一个新的积分空列表
    # scores = []
    
    for df1 in df[0]: # 按顺序遍历每一个名字
        a = 0
        # b = 0
        for 人员 in 项目["圈子昵称"]: # 遍历圈子中的每一个名字
            if 人员 == df1: # 最终名单得里的名字在圈子的名单中，就将圈子中的积分赋给a
                a = 项目.loc[人员, "积分"]
                
        积分.append(a) # 按遍历的顺序将a加到积分的列表中
        
    return 积分 # 返回列表

## 将积分以列表形式导出

In [13]:
所有项目积分 = {}

In [14]:
def 全部项目整合(dict, dict_reindexed):
    score = "积分"
    rein = "_reindexed"
    for key in dict:
        key_n = key + score
        key_rein = key + rein
        所有项目积分[key_n] = 单独项目赋分(dict_reindexed[key_rein], final_df_reindexed)

In [15]:
全部项目整合(data_dict, data_dict_reindexed)

In [16]:
len(所有项目积分["单词积分"])

19

In [18]:
# len(所有项目积分['记账积分'])

In [19]:
len(所有项目积分['跑步积分'])

19

In [20]:
len(所有项目积分['光盘积分'])
所有项目积分['光盘积分']

[0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 14, 13, 10, 9]

In [21]:
len(所有项目积分['练字积分'])
所有项目积分['练字积分']

[14, 13, 6, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## 在最终结果中的DataFrame中创建五个新的列，然后将各项目的积分加到列中

In [22]:
final_dataframe = final_df_reindexed

In [23]:
def dataframe_add_events_columns(data_score):
    for key in data_score:
        final_dataframe[key] = data_score[key]

In [24]:
dataframe_add_events_columns(所有项目积分)

In [26]:
final_dataframe

,0,练字积分,单词积分,跑步积分,光盘积分
0,,,,,
1034111911兰斯,1034111911兰斯,14,0,0,0
1024533380孙笑川,1024533380孙笑川,13,0,0,0
1044453937王八,1044453937王八,6,0,0,0
1034343407王雷,1034343407王雷,4,0,0,0
1024191981张全蛋,1024191981张全蛋,1,0,0,0
1024102265陈楞宇,1024102265陈楞宇,0,28,0,0
1034104114王晓为,1034104114王晓为,0,28,0,0
1024100594彭泽军,1024100594彭泽军,0,28,0,0
1034103034梁左先,1034103034梁左先,0,28,0,0


## 计算总积分

### 计算总积分的函数

In [27]:
events_score_key = []
for key in 所有项目积分:
    events_score_key.append(key)
events_score_key

['练字积分', '单词积分', '跑步积分', '光盘积分']

In [28]:
def final_score_sum(df, data_key):
    final_score = [] # 创建总积分列表
    
    for name in df[0]: # 遍历每一个名字
        a = 0 
        for events in data_key:
            a = a + df.loc[name, events]
        # a = df.loc[df1, "单词积分"] + df.loc[df1, "跑步积分"] + df.loc[df1, "记账积分"] + df.loc[df1, "练字积分"] + df.loc[df1, "光盘积分"]
        # 将每一个人的各项目积分按行求和
        final_score.append(a) # 按顺序加入到总积分列表后面
        
    return final_score # 返回总积分

### 计算总积分

In [29]:
final_score = final_score_sum(final_dataframe, events_score_key) # 调用函数

In [30]:
len(final_score)

19

### 将总积分加入到DataFrame中

In [31]:
final_dataframe["总积分"] = final_score # 创建新列，载入总积分
final_dataframe

,0,练字积分,单词积分,跑步积分,光盘积分,总积分
0,,,,,,
1034111911兰斯,1034111911兰斯,14,0,0,0,14
1024533380孙笑川,1024533380孙笑川,13,0,0,0,13
1044453937王八,1044453937王八,6,0,0,0,6
1034343407王雷,1034343407王雷,4,0,0,0,4
1024191981张全蛋,1024191981张全蛋,1,0,0,0,1
1024102265陈楞宇,1024102265陈楞宇,0,28,0,0,28
1034104114王晓为,1034104114王晓为,0,28,0,0,28
1024100594彭泽军,1024100594彭泽军,0,28,0,0,28
1034103034梁左先,1034103034梁左先,0,28,0,0,28


## 将最终结果导出到excel

In [32]:
final_dataframe.to_excel('总积分表.xlsx', index=False) # 导出，文件名为“总积分表.xlsx”